In [1]:
# import libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Join two CSV files and prep them for analysis
# Stats Data, https://github.com/blnkpagelabs/nflscraPy/releases/tag/Stats
# Game Results Data, https://github.com/blnkpagelabs/nflscraPy?tab=readme-ov-file#seasons

season_df = pd.read_csv("Season-2023 Week14.csv")
stats_df = pd.read_csv("Stats-2023 Week13.csv")

In [4]:
# drop unnecessary columns
season_df = season_df.drop(['boxscore_stats_link','tm_nano'], axis=1)
stats_df = stats_df.drop(['boxscore_stats_link','nano'], axis=1)


In [5]:
# Process data files to ensure they can be merged
unique_aliases_season = season_df['tm_name'].unique().tolist()
unique_aliases_stats = stats_df['name'].unique().tolist()

In [6]:
# Create an empty list to store mismatched values
mismatched_values = []

# Loop through each value in unique_aliases_season
for value in unique_aliases_season:
  # Check if the value exists in unique_aliases_stats
  if value not in unique_aliases_stats:
    # Add the value and its source list (unique_aliases_season) to mismatched_values
    mismatched_values.append((value, "unique_aliases_season"))

# Loop through each value in unique_aliases_stats (opposite direction)
for value in unique_aliases_stats:
  # Check if the value exists in unique_aliases_season (already checked in first loop)
  if value not in unique_aliases_season:
    # Add the value and its source list (unique_aliases_stats) to mismatched_values
    mismatched_values.append((value, "unique_aliases_stats"))

# Print the list of mismatched values and their source lists
if mismatched_values:
  # Print header
  print("Following values differ between the lists:")
  # Loop through mismatched_values and print each tuple
  for value, source in mismatched_values:
    print(f"\t- {value} (from {source})")
else:
  print("All values in both lists are the same.")


All values in both lists are the same.


In [7]:
#merge the dataframes
merged_df = season_df.copy()

#create columns in merged_df
# Define stat names (assuming they are not present in your data)
stat_cols = ['rush_att', 'rush_yds', 'rush_tds', 'pass_cmp', 'pass_att', 
             'pass_cmp_pct', 'pass_yds', 'pass_tds', 'pass_int', 'passer_rating', 
             'net_pass_yds', 'total_yds', 'times_sacked', 'yds_sacked_for', 
             'fumbles', 'fumbles_lost', 'turnovers', 'penalties', 'penalty_yds', 
             'first_downs', 'third_down_conv', 'third_down_att', 'third_down_conv_pct', 
             'fourth_down_conv', 'fourth_down_att', 'fourth_down_conv_pct', 'time_of_possession']

# Define a dictionary to map stat names to suffixes for home and away teams
stat_suffixes = {'_A': 'Away', '_H': 'Home'}

# Create new columns with suffixes for home and away teams, filled with NaN initially
for stat in stat_cols:
  merged_df[stat + '_A'] = np.nan
  merged_df[stat + '_H'] = np.nan

# Add labels to stat names using the dictionary
for stat, suffix in stat_suffixes.items():
  merged_df.rename(columns={stat: f"{suffix} {stat[:-2]}"}, inplace=True)

print("Placeholder columns for home/away team stats created.")



Placeholder columns for home/away team stats created.


In [8]:
# Create a list of columns to update
columns_to_update = ['rush_att', 'rush_yds', 'rush_tds', 'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds', 'pass_tds', 'pass_int', 'passer_rating', 'net_pass_yds', 'total_yds', 'times_sacked', 'yds_sacked_for', 'fumbles', 'fumbles_lost', 'turnovers', 'penalties', 'penalty_yds', 'first_downs', 'third_down_conv', 'third_down_att', 'third_down_conv_pct', 'fourth_down_conv', 'fourth_down_att', 'fourth_down_conv_pct', 'time_of_possession']

# Merge and update data for 'tm_name'
merged_df_tm = merged_df.merge(stats_df, left_on=['event_date', 'tm_name'], right_on=['event_date', 'name'], how='left', suffixes=('', '_tm'))
for column in columns_to_update:
    if column in merged_df_tm.columns:
        merged_df.loc[merged_df['tm_location'] == 'H', f"{column}_H"] = merged_df_tm.loc[merged_df['tm_location'] == 'H', column]
        merged_df.loc[merged_df['tm_location'] == 'A', f"{column}_A"] = merged_df_tm.loc[merged_df['tm_location'] == 'A', column]

# Merge and update data for 'opp_name'
merged_df_opp = merged_df.merge(stats_df, left_on=['event_date', 'opp_name'], right_on=['event_date', 'name'], how='left', suffixes=('', '_opp'))
for column in columns_to_update:
    if column in merged_df_opp.columns:
        merged_df.loc[merged_df['opp_location'] == 'H', f"{column}_H"] = merged_df_opp.loc[merged_df['opp_location'] == 'H', column]
        merged_df.loc[merged_df['opp_location'] == 'A', f"{column}_A"] = merged_df_opp.loc[merged_df['opp_location'] == 'A', column]

# Create a list of columns to drop, skipping the ones that don't exist
columns_to_drop = [f"{column}_tm" for column in columns_to_update if f"{column}_tm" in merged_df_tm.columns]
columns_to_drop.extend([f"{column}_opp" for column in columns_to_update if f"{column}_opp" in merged_df_opp.columns])
if 'name_tm' in merged_df_tm.columns:
    columns_to_drop.append('name_tm')
if 'name_opp' in merged_df_opp.columns:
    columns_to_drop.append('name_opp')

# Drop the unnecessary columns from merged_df
merged_df = merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns])

In [9]:
# create more features and fill in pass completion rate if missing
# calculate pass completion rate
merged_df.loc[merged_df['pass_cmp_pct_A'].isna(), 'pass_cmp_pct_A'] = merged_df['pass_cmp_A'] / merged_df['pass_att_A']
merged_df.loc[merged_df['pass_cmp_pct_H'].isna(), 'pass_cmp_pct_H'] = merged_df['pass_cmp_H'] / merged_df['pass_att_H']

# Define a dictionary to map column name suffixes to team names
team_suffixes = {'_A': 'Away', '_H': 'Home'}

# Iterate through team suffixes (A and H)
for suffix, team_name in team_suffixes.items():
  # Create the new column name for total plays
  tot_plays_col = f"tot_plays{suffix}"

  # Calculate the sum of rush attempts and pass attempts using string formatting
  merged_df[tot_plays_col] = merged_df[f'rush_att{suffix}'] + merged_df[f'pass_att{suffix}']

  # Print a message for clarity
  print(f"Created column '{tot_plays_col}' for total plays ({team_name} team).")

# Avoid division by zero errors with try-except blocks and handle potential missing data
for team_suffix in ('_A', '_H'):
  tot_plays_col = f"tot_plays{team_suffix}"
  total_yds_col = f"total_yds{team_suffix}"
  new_col_name = f"total_yds_per_play{team_suffix}"

  try:
    # Calculate total yards per play (handling division by zero)
    merged_df[new_col_name] = np.where(merged_df[tot_plays_col] > 0, 
                                       merged_df[total_yds_col] / merged_df[tot_plays_col], 
                                       np.nan)
    print(f"Created column '{new_col_name}' for total yards per play ({team_suffix} team).")
  except KeyError:
    print(f"Error: Columns 'tot_plays{team_suffix}' or 'total_yds{team_suffix}' might be missing.")


Created column 'tot_plays_A' for total plays (Away team).
Created column 'tot_plays_H' for total plays (Home team).
Created column 'total_yds_per_play_A' for total yards per play (_A team).
Created column 'total_yds_per_play_H' for total yards per play (_H team).


In [10]:
# convert tm_score and opp_score to points_A and points_H
merged_df['points_A'] = np.nan
merged_df['points_H'] = np.nan

# Define new column names for points
points_cols = {'H': 'points_H', 'A': 'points_A'}

# Iterate through rows in merged_df
for index, row in merged_df.iterrows():
  # Extract tm_location 
  tm_location = row.get('tm_location')

  # Check if tm_location exists and assign points based on location (if possible)
  if tm_location in points_cols:
    merged_df.at[index, points_cols[tm_location]] = row['tm_score']
  else:
    # Handle cases where tm_location is missing or not 'H' or 'A' (optional)
    pass

  # Repeat for opponent's location 
  opp_location = row.get('opp_location')
  if opp_location in points_cols:
    merged_df.at[index, points_cols[opp_location]] = row['opp_score']
  else:
    # Handle cases where opp_location is missing or not 'H' or 'A' (optional)
    print(f"Warning: Missing or invalid 'opp_location' value for row {index}. Points not assigned.")

In [11]:
# Add columns for differentials

# Calculate the difference between "total_yds_A" and "total_yds_H" and store it in a new column
merged_df['tot_yds_diff_A'] = merged_df['total_yds_A'] - merged_df['total_yds_H']
merged_df['tot_yds_diff_H'] = merged_df['total_yds_H'] - merged_df['total_yds_A']

# Calculate the difference between "rush_yds_A" and "rush_yds_H" and store it in a new column
merged_df['rush_yds_diff_A'] = merged_df['rush_yds_A'] - merged_df['rush_yds_H']
merged_df['rush_yds_diff_H'] = merged_df['rush_yds_H'] - merged_df['rush_yds_A']

# Add columns for turnover differential
# Calculate the difference between "turnovers_A" and "turnovers_H" and store it in a new column
merged_df['turnover_diff_A'] = merged_df['turnovers_A'] - merged_df['turnovers_H']
merged_df['turnover_diff_H'] = merged_df['turnovers_H'] - merged_df['turnovers_A']

# Calculate the difference between "time_of_possession_A" and "time_of_possession_H" and store it in a new column
merged_df['time_poss_diff_A'] = merged_df['time_of_possession_A'] - merged_df['time_of_possession_H']
merged_df['time_poss_diff_H'] = merged_df['time_of_possession_H'] - merged_df['time_of_possession_A']

# Calculate the difference between "total_yds_per_play_A" and "total_yds_per_play_H" and store it in a new column
merged_df['tot_yds_play_diff_A'] = merged_df['total_yds_per_play_A'] - merged_df['total_yds_per_play_H']
merged_df['tot_yds_play_diff_H'] = merged_df['total_yds_per_play_H'] - merged_df['total_yds_per_play_A']

# Calculate the difference between "third_down_conv_pct_A" and "third_down_conv_pct_H" and store it in a new column
merged_df['third_down_effic_diff_A'] = merged_df['third_down_conv_pct_A'] - merged_df['third_down_conv_pct_H']
merged_df['third_down_effic_diff_H'] = merged_df['third_down_conv_pct_H'] - merged_df['third_down_conv_pct_A']

# Calculate the difference between "Points_A" and "Points_H" and store it in a new column
merged_df['point_diff_A'] = merged_df['points_A'] - merged_df['points_H']
merged_df['point_diff_H'] = merged_df['points_H'] - merged_df['points_A']

# Encode Away team as -1 and Home team as +1
merged_df['away'] = -1
merged_df['home'] = 1

# Encode result of game. -1 for loss. +1 for win. 0 for tie.
# Add a new column named "Result_A" based on the values of "Point_diff_A"
merged_df['result_A'] = merged_df['point_diff_A'].apply(lambda x: 1 if x > 0 else (0 if x < 0 else 0))

# Add a new column named "Result_H" based on the values of "Point_diff_H"
merged_df['result_H'] = merged_df['point_diff_H'].apply(lambda x: 1 if x > 0 else (0 if x < 0 else 0))


In [12]:
# Create 'name_A' and 'name_H' columns based on 'tm_location'
merged_df['name_A'] = np.where(merged_df['tm_location'] == 'H', merged_df['opp_name'], merged_df['tm_name'])
merged_df['name_H'] = np.where(merged_df['tm_location'] == 'H', merged_df['tm_name'], merged_df['opp_name'])

In [13]:
# Create an empty list to store the rows for 'team_df'
team_rows = []

# Iterate over each row in 'merged_df'
for _, row in merged_df.iterrows():
    # Create a dictionary to store the team's data
    team_data = {
        'status': row['status'],
        'season': row['season'],
        'week': row['week'],
        'week_day': row['week_day'],
        'event_date': row['event_date'],
        'market': row['tm_market'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_market'],
        'name': row['tm_name'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_name'],
        'alias': row['tm_alias'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_alias'],
        'alt_market': row['tm_alt_market'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_alt_market'],
        'alt_alias': row['tm_alt_alias'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_alt_alias'],
        'location': row['tm_location'],
        'score': row['tm_score'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else row['opp_score'],
        'rush_att': row['rush_att_A'] if row['tm_location'] == 'A' else row['rush_att_H'],
        'rush_yds': row['rush_yds_A'] if row['tm_location'] == 'A' else row['rush_yds_H'],
        'rush_tds': row['rush_tds_A'] if row['tm_location'] == 'A' else row['rush_tds_H'],
        'pass_cmp': row['pass_cmp_A'] if row['tm_location'] == 'A' else row['pass_cmp_H'],
        'pass_att': row['pass_att_A'] if row['tm_location'] == 'A' else row['pass_att_H'],
        'pass_cmp_pct': row['pass_cmp_pct_A'] if row['tm_location'] == 'A' else row['pass_cmp_pct_H'],
        'pass_yds': row['pass_yds_A'] if row['tm_location'] == 'A' else row['pass_yds_H'],
        'pass_tds': row['pass_tds_A'] if row['tm_location'] == 'A' else row['pass_tds_H'],
        'pass_int': row['pass_int_A'] if row['tm_location'] == 'A' else row['pass_int_H'],
        'passer_rating': row['passer_rating_A'] if row['tm_location'] == 'A' else row['passer_rating_H'],
        'net_pass_yds': row['net_pass_yds_A'] if row['tm_location'] == 'A' else row['net_pass_yds_H'],
        'total_yds': row['total_yds_A'] if row['tm_location'] == 'A' else row['total_yds_H'],
        'times_sacked': row['times_sacked_A'] if row['tm_location'] == 'A' else row['times_sacked_H'],
        'yds_sacked_for': row['yds_sacked_for_A'] if row['tm_location'] == 'A' else row['yds_sacked_for_H'],
        'fumbles': row['fumbles_A'] if row['tm_location'] == 'A' else row['fumbles_H'],
        'fumbles_lost': row['fumbles_lost_A'] if row['tm_location'] == 'A' else row['fumbles_lost_H'],
        'turnovers': row['turnovers_A'] if row['tm_location'] == 'A' else row['turnovers_H'],
        'penalties': row['penalties_A'] if row['tm_location'] == 'A' else row['penalties_H'],
        'penalty_yds': row['penalty_yds_A'] if row['tm_location'] == 'A' else row['penalty_yds_H'],
        'first_downs': row['first_downs_A'] if row['tm_location'] == 'A' else row['first_downs_H'],
        'third_down_conv': row['third_down_conv_A'] if row['tm_location'] == 'A' else row['third_down_conv_H'],
        'third_down_att': row['third_down_att_A'] if row['tm_location'] == 'A' else row['third_down_att_H'],
        'third_down_conv_pct': row['third_down_conv_pct_A'] if row['tm_location'] == 'A' else row['third_down_conv_pct_H'],
        'fourth_down_conv': row['fourth_down_conv_A'] if row['tm_location'] == 'A' else row['fourth_down_conv_H'],
        'fourth_down_att': row['fourth_down_att_A'] if row['tm_location'] == 'A' else row['fourth_down_att_H'],
        'fourth_down_conv_pct': row['fourth_down_conv_pct_A'] if row['tm_location'] == 'A' else row['fourth_down_conv_pct_H'],
        'time_of_possession': row['time_of_possession_A'] if row['tm_location'] == 'A' else row['time_of_possession_H'],
        'points': row['points_A'] if row['tm_location'] == 'A' else row['points_H'],
        'tot_yds_diff': row['tot_yds_diff_A'] if row['tm_location'] == 'A' else row['tot_yds_diff_H'],
        'rush_yds_diff': row['rush_yds_diff_A'] if row['tm_location'] == 'A' else row['rush_yds_diff_H'],
        'turnover_diff': row['turnover_diff_A'] if row['tm_location'] == 'A' else row['turnover_diff_H'],
        'time_poss_diff': row['time_poss_diff_A'] if row['tm_location'] == 'A' else row['time_poss_diff_H'],
        'point_diff': row['point_diff_A'] if row['tm_location'] == 'A' else row['point_diff_H'],
        'away': row['away'] if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 0 and row['tm_location'] == 'H') else row['home'],
        'home': row['home'] if (row['result_H'] == 1 and row['tm_location'] == 'H') or (row['result_A'] == 0 and row['tm_location'] == 'A') else row['away'],
        'result': 1 if (row['result_A'] == 1 and row['tm_location'] == 'A') or (row['result_H'] == 1 and row['tm_location'] == 'H') else 0,
        'tot_plays': row['tot_plays_A'] if row['tm_location'] == 'A' else row['tot_plays_H'],
        'total_yds_per_play': row['total_yds_per_play_A'] if row['tm_location'] == 'A' else row['total_yds_per_play_H']

    }
    # Append the opponent's data to the list of rows
    team_rows.append(team_data)

# Create the 'team_df' dataframe from the list of rows
team_df = pd.DataFrame(team_rows)

In [14]:
# Create an empty list to store the rows for 'team_df'
opp_rows = []

# Iterate over each row in 'merged_df'
for _, row in merged_df.iterrows():
    # Create a dictionary to store the team's data
    opp_data = {
        'status': row['status'],
        'season': row['season'],
        'week': row['week'],
        'week_day': row['week_day'],
        'event_date': row['event_date'],
        'market': row['opp_market'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_market'],
        'name': row['opp_name'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_name'],
        'alias': row['opp_alias'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_alias'],
        'alt_market': row['opp_alt_market'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_alt_market'],
        'alt_alias': row['opp_alt_alias'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_alt_alias'],
        'location': row['opp_location'],
        'score': row['opp_score'] if (row['result_A'] == 0 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['tm_score'],
        'rush_att': row['rush_att_A'] if row['opp_location'] == 'A' else row['rush_att_H'],
        'rush_yds': row['rush_yds_A'] if row['opp_location'] == 'A' else row['rush_yds_H'],
        'rush_tds': row['rush_tds_A'] if row['opp_location'] == 'A' else row['rush_tds_H'],
        'pass_cmp': row['pass_cmp_A'] if row['opp_location'] == 'A' else row['pass_cmp_H'],
        'pass_att': row['pass_att_A'] if row['opp_location'] == 'A' else row['pass_att_H'],
        'pass_cmp_pct': row['pass_cmp_pct_A'] if row['opp_location'] == 'A' else row['pass_cmp_pct_H'],
        'pass_yds': row['pass_yds_A'] if row['opp_location'] == 'A' else row['pass_yds_H'],
        'pass_tds': row['pass_tds_A'] if row['opp_location'] == 'A' else row['pass_tds_H'],
        'pass_int': row['pass_int_A'] if row['opp_location'] == 'A' else row['pass_int_H'],
        'passer_rating': row['passer_rating_A'] if row['opp_location'] == 'A' else row['passer_rating_H'],
        'net_pass_yds': row['net_pass_yds_A'] if row['opp_location'] == 'A' else row['net_pass_yds_H'],
        'total_yds': row['total_yds_A'] if row['opp_location'] == 'A' else row['total_yds_H'],
        'times_sacked': row['times_sacked_A'] if row['opp_location'] == 'A' else row['times_sacked_H'],
        'yds_sacked_for': row['yds_sacked_for_A'] if row['opp_location'] == 'A' else row['yds_sacked_for_H'],
        'fumbles': row['fumbles_A'] if row['opp_location'] == 'A' else row['fumbles_H'],
        'fumbles_lost': row['fumbles_lost_A'] if row['opp_location'] == 'A' else row['fumbles_lost_H'],
        'turnovers': row['turnovers_A'] if row['opp_location'] == 'A' else row['turnovers_H'],
        'penalties': row['penalties_A'] if row['opp_location'] == 'A' else row['penalties_H'],
        'penalty_yds': row['penalty_yds_A'] if row['opp_location'] == 'A' else row['penalty_yds_H'],
        'first_downs': row['first_downs_A'] if row['opp_location'] == 'A' else row['first_downs_H'],
        'third_down_conv': row['third_down_conv_A'] if row['opp_location'] == 'A' else row['third_down_conv_H'],
        'third_down_att': row['third_down_att_A'] if row['opp_location'] == 'A' else row['third_down_att_H'],
        'third_down_conv_pct': row['third_down_conv_pct_A'] if row['opp_location'] == 'A' else row['third_down_conv_pct_H'],
        'fourth_down_conv': row['fourth_down_conv_A'] if row['opp_location'] == 'A' else row['fourth_down_conv_H'],
        'fourth_down_att': row['fourth_down_att_A'] if row['opp_location'] == 'A' else row['fourth_down_att_H'],
        'fourth_down_conv_pct': row['fourth_down_conv_pct_A'] if row['opp_location'] == 'A' else row['fourth_down_conv_pct_H'],
        'time_of_possession': row['time_of_possession_A'] if row['opp_location'] == 'A' else row['time_of_possession_H'],
        'points': row['points_A'] if row['opp_location'] == 'A' else row['points_H'],
        'tot_yds_diff': row['tot_yds_diff_A'] if row['opp_location'] == 'A' else row['tot_yds_diff_H'],
        'rush_yds_diff': row['rush_yds_diff_A'] if row['opp_location'] == 'A' else row['rush_yds_diff_H'],
        'turnover_diff': row['turnover_diff_A'] if row['opp_location'] == 'A' else row['turnover_diff_H'],
        'time_poss_diff': row['time_poss_diff_A'] if row['opp_location'] == 'A' else row['time_poss_diff_H'],
        'point_diff': row['point_diff_A'] if row['opp_location'] == 'A' else row['point_diff_H'],
        'away': row['away'] if (row['result_A'] == 1 and row['opp_location'] == 'A') or (row['result_H'] == 0 and row['opp_location'] == 'H') else row['home'],
        'home': row['home'] if (row['result_H'] == 1 and row['opp_location'] == 'H') or (row['result_A'] == 0 and row['opp_location'] == 'A') else row['away'],
        'result': 1 if (row['result_A'] == 1 and row['opp_location'] == 'A') or (row['result_H'] == 1 and row['opp_location'] == 'H') else 0,
        'tot_plays': row['tot_plays_A'] if row['opp_location'] == 'A' else row['tot_plays_H'],
        'total_yds_per_play': row['total_yds_per_play_A'] if row['opp_location'] == 'A' else row['total_yds_per_play_H']

    }
    # Append the opponent's data to the list of rows
    team_rows.append(opp_data)

# Create the 'team_df' dataframe from the list of rows
team_df = pd.DataFrame(team_rows)

In [15]:
# Calculate season to date team averages

# Create an empty list to store the dataframes
dataframes = []

# Get the maximum number of weeks in the 'Week' column
max_weeks = team_df['week'].max()

# Iterate from weeks 12 to the maximum number of weeks
for i in range(13, 14):
    # Create a new dataframe by filtering the original team_df
    filtered_df = team_df[(team_df['week'] >= 1) & (team_df['week'] <= i)].copy()

    # Select the specified columns from the filtered dataframe
    new_df = filtered_df[['name', 'rush_yds_diff', 'turnover_diff', 'time_poss_diff', 'point_diff', 'tot_yds_diff','away','home']]

    # Append the new dataframe to the list
    dataframes.append(new_df)

# Assign the dataframes to variables dynamically
for i, dataframe in enumerate(dataframes, start=13):
    globals()[f"std{i}_df"] = dataframe

# Calculate the average values for each 'name' in the dataframes
team_avg_dfs = []
for dataframe in dataframes:
    avg_df = dataframe.groupby('name').mean().reset_index()
    team_avg_dfs.append(avg_df)

# Assign the average dataframes to variables dynamically
for i, avg_df in enumerate(team_avg_dfs, start=13):
    globals()[f"team_avg_wk{i}"] = avg_df

In [16]:
# Save merged_df to a CSV file (excluding index by default)
filtered_df.to_csv("filtered_df.csv", index=False)

print("Successfully created 'filtered_df.csv'.")

Successfully created 'filtered_df.csv'.


In [17]:
# create the head to head matchups and load the season to date averages as the Kahn features.

# Create an empty list to store the dataframes
dataframes = []

# Iterate over weeks 13 to max_weeks
for week in range(13, max_weeks + 1):
    # Filter the dataframe for rows where 'Week' column is equal to the current week
    filtered_df = merged_df[merged_df['week'] == week].copy()  # Use .copy() to avoid SettingWithCopyWarning
    
    # Select the specified columns from the filtered dataframe
    selected_columns = ['week', 'name_A','name_H', 'result_A','result_H', 'points_A', 'points_H', 'away', 'home']
    new_df = filtered_df[selected_columns].copy()  # Make a copy to avoid SettingWithCopyWarning
    
    # Add the new columns with NaN values using .loc and appending '_A' or '_H' to each column name
    new_columns = ['rush_yds_diff_A', 'turnover_diff_A', 'time_poss_diff_A', 'Point_diff_A', 'tot_yds_diff_A',
                   'rush_yds_diff_H', 'turnover_diff_H', 'time_poss_diff_H', 'Point_diff_H', 'tot_yds_diff_H']
    for col in new_columns:
        new_df.loc[:, col] = np.nan  # Use .loc to assign NaN values to the new columns
    
    # Append the new dataframe to the list
    dataframes.append(new_df)

# Assign the dataframes to variables dynamically
for i, dataframe in enumerate(dataframes, start=14):
    globals()[f"week{i}_df"] = dataframe

In [18]:
df_list = dataframes[1:]  # Assuming dataframes list starts from week 13
team_avg_list = [globals()[f"team_avg_wk{i}"] for i in range(13, max_weeks)]

# Iterate over the dataframes
for i in range(len(df_list)):
    df = df_list[i]
    team_avg_df = team_avg_list[i]
# Populate columns for away team
    df['rush_yds_diff_A'] = df['name_A'].map(team_avg_df.set_index('name')['rush_yds_diff'])
    df['turnover_diff_A'] = df['name_A'].map(team_avg_df.set_index('name')['turnover_diff'])
    df['time_poss_diff_A'] = df['name_A'].map(team_avg_df.set_index('name')['time_poss_diff'])
    df['Point_diff_A'] = df['name_A'].map(team_avg_df.set_index('name')['point_diff'])
    df['tot_yds_diff_A'] = df['name_A'].map(team_avg_df.set_index('name')['tot_yds_diff'])

    # Populate columns for home team
    df['rush_yds_diff_H'] = df['name_H'].map(team_avg_df.set_index('name')['rush_yds_diff'])
    df['turnover_diff_H'] = df['name_H'].map(team_avg_df.set_index('name')['turnover_diff'])
    df['time_poss_diff_H'] = df['name_H'].map(team_avg_df.set_index('name')['time_poss_diff'])
    df['Point_diff_H'] = df['name_H'].map(team_avg_df.set_index('name')['point_diff'])
    df['tot_yds_diff_H'] = df['name_H'].map(team_avg_df.set_index('name')['tot_yds_diff'])
 # Separate features and targets
    X_test = df[['tot_yds_diff_A', 'rush_yds_diff_A', 'time_poss_diff_A', 'turnover_diff_A',
                 'tot_yds_diff_H', 'rush_yds_diff_H', 'time_poss_diff_H', 'turnover_diff_H']].values
    y_test = df['result_H'].values  # Use 'result_H' as the target variable
    name_A = df['name_A'].values
    name_H = df['name_H'].values

    # Create DMatrix for XGBoost
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define XGBoost parameters
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': 42
    }

    # Train XGBoost model
    model = xgb.train(params, dtest)

    # Make predictions
    y_pred = model.predict(dtest)
    y_pred = (y_pred >= 0.5).astype(int)  # Convert probabilities to class labels

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy for Week {i+14}: {accuracy:.4f}')
    print('Predictions:')
    for j in range(len(y_test)):
        print(f'Away Team: {name_A[j]}, Home Team: {name_H[j]}, Actual Winner: {y_test[j]}, Predicted Winner: {y_pred[j]}')

    print()  # Add a blank line between weeks for readability

Test Accuracy for Week 14: 0.9333
Predictions:
Away Team: Packers, Home Team: Giants, Actual Winner: 1, Predicted Winner: 1
Away Team: Titans, Home Team: Dolphins, Actual Winner: 0, Predicted Winner: 0
Away Team: Seahawks, Home Team: 49ers, Actual Winner: 1, Predicted Winner: 1
Away Team: Broncos, Home Team: Chargers, Actual Winner: 0, Predicted Winner: 0
Away Team: Rams, Home Team: Ravens, Actual Winner: 1, Predicted Winner: 1
Away Team: Vikings, Home Team: Raiders, Actual Winner: 0, Predicted Winner: 1
Away Team: Texans, Home Team: Jets, Actual Winner: 1, Predicted Winner: 1
Away Team: Panthers, Home Team: Saints, Actual Winner: 1, Predicted Winner: 1
Away Team: Bills, Home Team: Chiefs, Actual Winner: 0, Predicted Winner: 0
Away Team: Eagles, Home Team: Cowboys, Actual Winner: 1, Predicted Winner: 1
Away Team: Jaguars, Home Team: Browns, Actual Winner: 1, Predicted Winner: 1
Away Team: Colts, Home Team: Bengals, Actual Winner: 1, Predicted Winner: 1
Away Team: Lions, Home Team: Bear